In [1]:
import os,sys,inspect
import numpy as np
import torch
import torch.nn as nn
import cv2
import torch.nn.functional as F

#os.environ["CUDA_VISIBLE_DEVICES"]="1"

from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import cm

from lib.hybridnet.modules.v2vnet.repro_layer import ReprojectionLayer
from lib.dataset.dataset3D import Dataset3D
from lib.hybridnet.modules.efficienttrack.model import EfficientTrackBackbone
import lib.hybridNet.modules.efficienttrack.darkpose as darkpose
from lib.hybridnet.utils import ReprojectionTool
from lib.hybridnet.modules.v2vnet.model import V2VNet
from lib.config.project_manager import ProjectManager
from lib.hybridnet.model import HybridNetBackbone

project = ProjectManager()
project.load('12Cam_wo_Ben')
cfg = project.get_cfg()


384
Successfully loaded project 12Cam_wo_Ben!


In [5]:
test_set = Dataset3D(cfg = cfg, set='val')
hybridNet = HybridNetBackbone(cfg, test_set.coco.dataset['calibration']['intrinsics'], 
                        test_set.coco.dataset['calibration']['extrinsics']).cuda()

weights = '/home/trackingsetup/Documents/Vortex/projects/12Cam_wo_Ben/models/vortex/Run_20211011-140132/Vortex-d_60.pth'
hybridNet.load_state_dict(torch.load(weights), strict = True)

reproTool = ReprojectionTool('Camera_T', test_set.root_dir, test_set.coco.dataset['calibration']['intrinsics'], test_set.coco.dataset['calibration']['extrinsics'])

hybridNet.requires_grad_(False)
hybridNet.eval()
hybridNet = hybridNet.cuda()




line_idxs = [[0,1], [1,2], [2,3], [4,5], [5,6], [6,7], [8,9], [9,10], [10,11], [12,13], [13,14], [14,15], [16,17], [17,18], [18,19], [3,7], [7,11], [11,15], [3,21], [7,21],[11,22], [15,22],[21,22], [18,15], [19,22]]

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!


In [6]:
name = "Ben"

pointsNet = []
points2DNet = []
pointsGT = []
tot_errors_hm3d = []
tot_errors_class = []
joint_lengths_hm3d = [[] for i in range(15)]
joint_lengths_class = [[] for i in range(15)]

print (len(test_set.image_ids))
for item in range(len(test_set.image_ids)):
    image_info = test_set.coco.loadImgs(test_set.image_ids[item])[0]
    print(image_info['file_name'].split("/")[0])
    if image_info['file_name'].split("/")[0] == name:
        sample = test_set.__getitem__(item)
        keypoints3D = sample[1]
        imgs = torch.from_numpy(np.array(sample[0])).cuda()
        imgs_p = torch.unsqueeze(imgs.permute(0,3,1,2).float(),0)
        centerHM = torch.cuda.IntTensor(sample[2])
        center3D = torch.cuda.FloatTensor(sample[3])

        heatmap3D, heatmaps_padded, points3D_net = hybridNet(imgs_p,torch.unsqueeze(centerHM,0),  torch.unsqueeze(center3D,0))
        preds, maxvals = darkpose.get_final_preds(heatmaps_padded[0].clamp(0,255).cpu().numpy(), None)
        preds *= 2

        points3D_rec = []
        for i in range(len(preds[0])):
            camsToUse = []
            for cam in range(12):
                if maxvals[cam][i] > 30:
                    camsToUse.append(cam)
            point = reproTool.reconstructPoint(np.transpose(preds[:,i,:]), camsToUse)
            points3D_rec.append(point)


        heatmap3D = heatmap3D.cpu().numpy()[0]
        center3D = center3D.cpu().numpy()
        points3D_net = points3D_net[0].cpu().detach().numpy()

        for i,idx in enumerate(line_idxs[0:15]):
            joint_lengths_hm3d[i].append(np.linalg.norm(np.array(points3D_net[idx[0]]-points3D_net[idx[1]])))
            joint_lengths_class[i].append(np.linalg.norm(np.array(keypoints3D[idx[0]]-keypoints3D[idx[1]])))

        pointsNet.append(points3D_net)
        points2DNet.append(points3D_rec)
        pointsGT.append(keypoints3D)

        errors_3dHM = []
        errors_Std = []
        for i in range(23):
            if i != 20:
                errors_3dHM.append(np.linalg.norm(np.array(points3D_net[i])-keypoints3D[i]))
                errors_Std.append(np.linalg.norm(np.array(points3D_rec[i])-keypoints3D[i]))
        print ('3DHM:', np.mean(np.array(errors_3dHM)))
        print ('Std:', np.mean(np.array(errors_Std)))

        tot_errors_hm3d.append(np.mean(np.array(errors_3dHM)))
        tot_errors_class.append(np.mean(np.array(errors_Std)))

92
Ben
3DHM: 3.003151670327151
Std: 8.658485767105061
Ben
3DHM: 2.4535699125123673
Std: 6.716976424228724
Ben
3DHM: 3.3820823054675055
Std: 7.657779862148365
Ben
3DHM: 3.3173276532199325
Std: 8.03746727492989
Ben
3DHM: 3.273671795817016
Std: 11.367218686456447
Ben
3DHM: 2.63137920829146
Std: 6.778030787825606
Ben
3DHM: 4.918309523850861
Std: 15.778246312111435
Ben
3DHM: 4.673987378684299
Std: 16.7346310990697
Ben
3DHM: 4.499260712624579
Std: 8.827297570392973
Ben
3DHM: 3.3573843718559995
Std: 8.705562752459887
Ben
3DHM: 4.8767442083718215
Std: 10.606323520874149
Ben
3DHM: 12.076237746061093
Std: 16.567164779067472
Ben
3DHM: 4.1379800935984585
Std: 7.865339351728756
Ben
3DHM: 3.7711626797676754
Std: 6.583761204886044
Ben
3DHM: 4.595946301628358
Std: 11.515550083372386
Ben
3DHM: 3.1006318260525507
Std: 7.67644483183324
Ben
3DHM: 3.554062573463949
Std: 9.804144717966485
Ben
3DHM: 3.610253292033721
Std: 9.553667117043453
Ben
3DHM: 4.33640903408391
Std: 12.653360623653391
Ben
3DHM: 3.767731

wo_name = 'Matthias'
print ("Var HM3D:", np.mean(np.var(np.array(joint_lengths_hm3d), axis = 1)))
print ("Var Class:", np.mean(np.var(np.array(joint_lengths_class), axis = 1)))
print ("Total 3DHM:", np.mean(tot_errors_hm3d))
print ("Total Classic:", np.mean(tot_errors_class))

from numpy import savetxt
savetxt('csvs/points3DNet_wo_' + wo_name + '.csv', np.array(pointsNet).reshape((-1, 23*3)), delimiter=',')
savetxt('csvs/points2DNet_wo_' + wo_name + '.csv', np.array(points2DNet).reshape((-1, 23*3)), delimiter=',')
savetxt('csvs/pointsGT_wo_' + wo_name + '.csv', np.array(pointsGT).reshape((-1, 23*3)), delimiter=',')